In [1]:
import numpy as np
import matplotlib.pyplot as plt

## Part I

In [2]:
# 读取数据
fn = 'test1.txt'
fn = 'input.txt'

data = []
with open(fn) as f:
    for line in f:
        lhs, rhs = line.strip().split("~")
        lhs = list(map(int, lhs.strip().split(",")))
        rhs = list(map(int, rhs.strip().split(",")))
        data.append([lhs, rhs])
# data = sorted(data, key=lambda x: min(x[0][2],x[1][2]))
print(*data[:10], sep="\n")

[[7, 3, 85], [9, 3, 85]]
[[2, 8, 123], [4, 8, 123]]
[[8, 2, 69], [8, 2, 72]]
[[3, 1, 280], [3, 4, 280]]
[[4, 1, 6], [4, 2, 6]]
[[7, 1, 156], [7, 3, 156]]
[[1, 5, 105], [4, 5, 105]]
[[5, 8, 69], [5, 8, 69]]
[[5, 0, 255], [5, 3, 255]]
[[2, 8, 120], [3, 8, 120]]


In [3]:
### 每个方向上的最大最小值
minx, maxx = min([min(p1[0],p2[0]) for p1, p2 in data]), max([max(p1[0],p2[0]) for p1, p2 in data])
miny, maxy = min([min(p1[1],p2[1]) for p1, p2 in data]), max([max(p1[1],p2[1]) for p1, p2 in data])
minz, maxz = min([min(p1[2],p2[2]) for p1, p2 in data]), max([max(p1[2],p2[2]) for p1, p2 in data])
print((minx,maxx), (miny,maxy), (minz, maxz))

(0, 9) (0, 9) (1, 299)


In [4]:
# 给定一块砖的两个端点坐标, 返回它包含的 cubes
def get_cubes(bk):
    x1,y1,z1 = bk[0]
    x2,y2,z2 = bk[1]
    t = None
    cubes = []
    if z1 != z2: # 垂直的
        z1, z2 = min(z1,z2),max(z1,z2)
        for zi in range(z1,z2+1):
            cubes.append([(x1,y1,zi)])
        t = "z"
    elif x1 != x2: # x 方向的
        x1, x2 = min(x1,x2),max(x1,x2)
        for xi in range(x1,x2+1):
            cubes.append((xi,y1,z1))
        cubes = [cubes]
        t = "x"
    elif y1 != y2: # y 方向的
        y1, y2 = min(y1,y2),max(y1,y2)
        for yi in range(y1,y2+1):
            cubes.append((x1,yi,z1))
        t = "y"
        cubes = [cubes]
    else:  # single cube
        cubes.append([(x1,y1,z1)])
        t = "z"
    return cubes, t

In [5]:
# 把每块砖打散为 cubes
data, kind = list(zip(*[get_cubes(bk) for bk in data]))
data = list(data)
kind = list(kind)
print(len(data))
print(*data[:10], sep="\n")

1203
[[(7, 3, 85), (8, 3, 85), (9, 3, 85)]]
[[(2, 8, 123), (3, 8, 123), (4, 8, 123)]]
[[(8, 2, 69)], [(8, 2, 70)], [(8, 2, 71)], [(8, 2, 72)]]
[[(3, 1, 280), (3, 2, 280), (3, 3, 280), (3, 4, 280)]]
[[(4, 1, 6), (4, 2, 6)]]
[[(7, 1, 156), (7, 2, 156), (7, 3, 156)]]
[[(1, 5, 105), (2, 5, 105), (3, 5, 105), (4, 5, 105)]]
[[(5, 8, 69)]]
[[(5, 0, 255), (5, 1, 255), (5, 2, 255), (5, 3, 255)]]
[[(2, 8, 120), (3, 8, 120)]]


In [6]:
# 以 xyz 坐标为 key, 构建一个数据库, 看这个 cube 被哪块砖占据了
db = dict()
for i, bk in enumerate(data):
    for row in bk:
        for cube in row:
            db[cube] = i
len(db)

3580

In [7]:
# 检查能够向下移动, 如果能就移动, 并更新 data 和 db
def fall(idx):
    global data
    global db
    
    is_below_empty = True
    while is_below_empty:  # 假设可以向下移动
        bk = data[idx]
        btrow = bk[0]   # 砖块的最底下的一层
        # 首先检查是不是真的可以向下移动
        for cube in btrow:
            x, y, z = cube
            if z == 1:   # 在地板上了, 不用动
                return False
            else:
                below_this_cube = x, y, z-1
                if below_this_cube in db and db[below_this_cube] is not None:    # 下面有东西挡住了
                    # nb[idx].add(db[below_this_cube])   # 记录下联系
                    is_below_empty = False
        # 如果真的可以, 就向下移动一格
        if is_below_empty:
            ## 更新砖块的位置
            new_bk = [[(x,y,z-1) for x,y,z in row] for row in bk ]
            ## data
            # print(data[idx]); print(new_bk)
            data[idx] = new_bk
            ## 更新 db
            for row in bk:
                for cb in row:
                    db.pop(cb)
            for row in new_bk:
                for cb in row:
                    db[cb] = idx
        
    return is_below_empty

In [8]:
# 每一层的每一个格子, 逐个检查
for zi in range(1, maxz+1):
    for xi in range(maxx+1):
        for yi in range(maxy+1):
            cb = (xi, yi, zi)
            if cb in db and db[cb] is not None: # 被某块砖占据了
                idx = db[cb]   # 砖块的 id
                fall(idx)      # 尝试向下移动
print(*data[:10], sep="\n")

[[(7, 3, 47), (8, 3, 47), (9, 3, 47)]]
[[(2, 8, 67), (3, 8, 67), (4, 8, 67)]]
[[(8, 2, 37)], [(8, 2, 38)], [(8, 2, 39)], [(8, 2, 40)]]
[[(3, 1, 138), (3, 2, 138), (3, 3, 138), (3, 4, 138)]]
[[(4, 1, 3), (4, 2, 3)]]
[[(7, 1, 82), (7, 2, 82), (7, 3, 82)]]
[[(1, 5, 54), (2, 5, 54), (3, 5, 54), (4, 5, 54)]]
[[(5, 8, 37)]]
[[(5, 0, 123), (5, 1, 123), (5, 2, 123), (5, 3, 123)]]
[[(2, 8, 66), (3, 8, 66)]]


In [9]:
# supported: [supporter]
nb = {i:set() for i, bk in enumerate(data)}
print(nb[0])

set()


In [10]:
# supported by whom?
for idx, bk in enumerate(data):
    bk = data[idx]
    btrow = bk[0]   # 砖块的最底下的一层
    is_below_empty = True  # 假设
    for cube in btrow:
            x, y, z = cube
            if z == 1:   # 在地板上了, 不用动
                is_below_empty = False
                break
            else:
                below_this_cube = x, y, z-1
                if below_this_cube in db and db[below_this_cube] is not None:    # 下面有东西挡住了
                    nb[idx].add(db[below_this_cube])   # 记录下联系
                    is_below_empty = False
    if is_below_empty:
        print("something is wrong", idx, bk)

In [11]:
allidx = set()
supporting_idx = set()

for idx, _ in enumerate(data):
    allidx.add(idx)                                 # 所有的 idx
    supporting_idx = supporting_idx.union(nb[idx])  # supporters

leaf_idx = allidx - supporting_idx      # 如果没有 support 任何其他的, 那它是一格 leaf (最顶端的)
                                        # leaf node 都是可以被安全撤掉的

count = 0
# 如果一块砖 (1) support A (2) A is supoorteb by at least 2 -> count
for idx in supporting_idx:
    for jdx, bk in enumerate(data):    # 检查它 support 了谁
        issafe = True
        if idx in nb[jdx] and len(nb[jdx]) == 1:  # support 了 jdx, 而且是独立 support, 就不能撤掉
            issafe = False
            break
    count += 1 if issafe else 0
            
count += len(leaf_idx)
print(count)

386


## Part II

In [12]:
# 构建一个 graph: supporting idx: [supported idx]
# 对比 nb: supported idx: [supoorting idx]
nb1 = {i:set() for i, bk in enumerate(data)}
for idx in allidx:
    for jdx in nb[idx]:
        nb1[jdx].add(idx)

In [14]:
# 检查一块砖由此往上是不是一棵独立的 tree
def is_root(idx):
    visited = set()
    queue = [idx]
    while queue:
        jdx = queue.pop(0)
        visited.add(jdx)
        supported = nb1[jdx]  # 所有被它 support 的 bricks
        for kdx in supported:
            supporters = nb[kdx]
            for hdx in supporters:
                if hdx not in visited:
                    return False, set()
            queue.append(kdx)
    return True, visited

In [19]:
# 检查每一个 idx 是不是 root of a tree
# 如果是, 则撤掉它掉落的砖块就是这个 tree 上所有节点的数量 - 1 (它自己)
is_root_of_tree = {i:None for i, bk in enumerate(data)}

for idx in allidx:
    yesorno, visited = is_root(idx)
    is_root_of_tree[id] = yesorno, visited

In [20]:
# 类似的思路, 检查一块砖撤掉之后会掉落多少
def count_falling(idx):
    visited = set()
    queue = [idx]
    while queue:
        jdx = queue.pop(0)
        visited.add(jdx)
        supported = nb1[jdx]  # 所有被它 support 的 bricks
        for kdx in supported: # 逐个检查这些 supported bricks
            supporters = nb[kdx]   # 看每个 supported bricks 都有哪些 supporters
            also_supported_by_others = False
            for hdx in supporters: # 看看这些 supporters 是不是已经 visited 过的 bricks
                if hdx not in visited:  # 说明这块砖有其他支撑
                    also_supported_by_others = True
                    break
            if not also_supported_by_others:
                queue.append(kdx)
    return visited

In [22]:
res = 0
for idx in allidx:
    visited = count_falling(idx)
    res += len(visited)-1
print(res)


39933
